# Customer Churn Prediction

Customer churn or customer attrition is a tendency of clients or customers to abandon a brand and stop being paying clients of a particular business or organization. The percentage of customers discontinuing using a company’s services or products during a specific period is called a customer churn rate. Several bad experiences (or just one) are enough, and a customer may quit. And if a large chunk of unsatisfied customers churn at a time interval, both material losses and damage to reputation would be enormous.

Using machine learning techniques, financial services companies use their customer data to identify behavior patterns of potential churners, classify these at-risk customers, and take appropriate actions to gain their trust and increase their retention rate. The task is to classify customers as churners or non-churners (binary classification) based on the organization’s historical data sources.

To perform the classification task, you can use machine learning classification algorithms like Logistic regression, Naive Bayes Classifier, Tree-based algorithms, Random Forest, etc. To increase efficiency, you can use advanced algorithms like XGBoost, LightGBM, or CatBoost. Use the accuracy metrics to compare the performance of the different models. You can use the Bank Customers Dataset for Churn to practice this project.

#### Train and Evaluate the Neural Network Model

1. Train (fit) the neural network model. Use the training data and set 100 epochs.

2. Evaluate the model using the test data to determine its loss and accuracy.


## References:

[Keras Sequential model](https://keras.io/api/models/sequential/)

[Keras Dense module](https://keras.io/api/layers/core_layers/dense/)

[Keras evaluate](https://keras.io/api/models/model_training_apis/)

[SKLearn OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)


In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

---

## Preprocess the data.

### Step 1: Read the `Churn_Modelling.csv` file from the Resources folder and create a DataFrame.

In [2]:
# Read the HR-Employee-Attrition.csv file from the Resources folder into a Pandas DataFrame
customer_df = pd.read_csv(Path("./Resources/Churn_Modelling.csv"))
customer_df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### Step 2: Review the resulting DataFrame. Check the data type associated with each column to identify categorical and non-categorical variables.

> **Hint** Recall that categorical variables have an `object` data type.


In [3]:
# Review the DataFrame
display(customer_df.head())

# Review the data types associated with the columns
display(customer_df.dtypes)
display(customer_df.shape)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

(10000, 14)

### Step 3: Create a list of categorical variables.

In [4]:
# Create a list of categorical variables 
categorical_variables = list(customer_df.dtypes[customer_df.dtypes == "object"].index)
categorical_variables = 

SyntaxError: invalid syntax (985995679.py, line 3)

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(customer_df[categorical_variables])

# Create a DataFrame with the encoded variables
# The column names should match those of the encoded variables
encoded_df = pd.DataFrame(encoded_data,columns = enc.get_feature_names_out(categorical_variables))
encoded_df

NameError: name 'categorical_variables' is not defined

In [6]:
# Create a DataFrame with the columnns containing numerical variables from the original dataset
numerical_variables_df = encoded_df.drop(columns = categorical_variables)

# Using the Pandas concat function, combine the DataFrames the contain the encoded categorical data and the numerical data
attition_df = pd.concat([numerical_variables_df,encoded_df],axis=1)

# Reveiw the DataFrame
attition_df.head()

NameError: name 'encoded_df' is not defined

In [7]:
# Define the target set y using the Attrition_Yes column
y = attition_df["Attrition_Yes"]


# Define features set X by selecting all columns but Attrition_Yes and Attrition_No
X = attition_df.drop(columns=["Attrition_Yes", "Attrition_No"])

# Review the features DataFrameb
display(y.shape)
display(X.shape)

NameError: name 'attition_df' is not defined

### Step 7: Create the training and testing sets using the `train_test_split` function from scikit-learn.

In [ ]:
# Split the data into training and testing datasets
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Create a Neural Network Model to Predict Employee Attrition

### Step 1: Use the optimization techniques presented in this lesson to create a deep neural network model with two hidden layers.

In [ ]:
# Define the the number of inputs (features) to the model (i.e. 55 = columns of X)
number_input_features = len(X_train.iloc[0])

# Define the number of hidden nodes for the first hidden layer
# Use the mean of the number of input features plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer1 =  (number_input_features + 1) // 2 

# Define the number of hidden nodes for the second hidden layer
# Use the mean of the number of hidden nodes in the first hidden layer plus the number of output nurons
# Use the Python floor division (//) to return the quotent
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Create the Sequential model instance
nn = Sequential()

# Add the first hidden layer specifying the number of inputs, the number of hidden nodes, and the activation function
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Add the second hidden layer specifying the number of hidden nodes and the activation function
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
# This model will predict a binary output. So, add an output layer with one neuron, and use the `sigmoid` activation function.
nn.add(Dense(units=1, activation="sigmoid"))

# Display the Sequential model summary
nn.summary()

### Step 2: Compile the model. Use the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` metric.

In [ ]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

### Step 3: Train (fit) the neural network model. Use the training data and set 100 epochs.


In [ ]:
# Fit the model using 100 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

### Step 4: Evaluate the model using the test data to determine its loss and accuracy.

In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")